# Chapter 2. Pure Python Optimizations

Buying a larger server or microoptimizing can work for some time, but achieving better scaling through algorithmic improvement can solve the problem once and for all.

## 2.1 Useful algorithms and data structures

Algorithm running times can be classified according to their computational complexity, a characterization of the resources required to perform a task. Such classification is expressed through the Big-O notation, an upper bound on the operations required to execute the task, which usually depends on the input size

For example, incrementing each element of a list can be implemented using a for loop, as follows:

In [ ]:
input = list(range(10))
for i, _ in enumerate(input):
        input[i] += 1

If the operation does not depend on the size of the input (for example, accessing the first element of a list), the algorithm is said to take constant, or O(1), time. This means that, no matter how much data we have, the time to run the algorithm will always be the same. In this simple algorithm, the input[i] += 1 operation will be repeated 10 times, which is the size of the input. If we double the size of the input array, the number of operations will increase proportionally. Since the number of operations is proportional to the input size, this algorithm is said to take O(N) time, where N is the size of the input array.

### Lists and deques

Python lists are ordered collections of elements and, in Python, are implemented as resizable arrays. An array is a basic data structure that consists of a series of contiguous memory locations, and each location contains a reference to a Python object.

Lists shine in accessing, modifying, and appending elements. Accessing or modifying an element involves fetching the object reference from the appropriate position of the underlying array and has complexity O(1). Appending an element is also very fast. When an empty list is created, an array of fixed size is allocated and, as we insert elements, the slots in the array are gradually filled up. Once all the slots are occupied, the list needs to increase the size of its underlying array, thus triggering a memory reallocation that can take O(N) time. Nevertheless, those memory allocations are infrequent, and the time complexity for the append operation is referred to as amortized O(1) time.

The list operations that may have efficiency problems are those that add or remove elements at the beginning (or somewhere in the middle) of the list. When an item is inserted, or removed, from the beginning of a list, all the subsequent elements of the array need to be shifted by a position, thus taking O(N) time.

*list.pop(), O(1)*

*list.pop(0), O(N)*

*list.append(1), O(1)*

*list.insert(0,1), O(N)*

In some cases, it is necessary to efficiently perform insertion or removal of elements both at the beginning and at the end of the collection. Python provides a data structure with those properties in the collections.deque class. The word deque stands for double-ended queue because this data structure is designed to efficiently put and remove elements at the beginning and at the end of the collection, as it is in the case of queues. In Python, deques are implemented as doubly-linked lists.

*deque.pop(), O(1)*

*deque.popleft(), O(1)*

*deque.append(1), O(1)*

*deque.appendleft(1), O(1)*

*deque[int(N/2)], O(N)*

Searching for an item in a list is generally a O(N) operation and is performed using the list.index method. A simple way to speed up searches in lists is to keep the array sorted and perform a binary search using the bisect module.

The bisect module allows fast searches on sorted arrays. The bisect.bisect function can be used on a sorted list to find the index to place an element while maintaining the array in sorted order.

In the following example, we can see that if we want to insert the 3 element in the array while keeping collection in sorted order, we should put 3 in the third position (which corresponds to index 2):

In [ ]:
insert bisect
collection = [1, 2, 4, 5, 6]
bisect.bisect(collection, 3)
# Result: 2

This function uses the binary search algorithm that has O(log(N)) running time. Such a running time is exceptionally fast, and basically means that your running time will increase by a constant amount every time you double your input size. This means that if, for example, your program takes 1 second to run on an input of size 1000, it will take 2 seconds to process an input of size 2000, 3 seconds to process an input of size 4000, and so on.

If the value we are trying to insert is already present in the list, the bisect.bisect function will return the location after the already present value. Therefore, we can use the bisect.bisect_left variant, which returns the correct index

In [ ]:
def index_bisect(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect.bisect_left(a, x)
    if i != len(a) and a[i] == x:
    return i
    raise ValueError

*list.index(a), O(N)*

*index_bisect(list, a), O(log(N))*

### Dictionaries

Dictionaries are implemented as hash maps and are very good at element insertion, deletion, and access; all these operations have an average O(1) time complexity

In Python versions up to 3.5, dictionaries are unordered collections. Since Python 3.6, dictionaries are capable of maintaining their elements by order of insertion.

A **hash map** is a data structure that associates a set of **key-value pairs**. The principle behind hash maps is to assign a specific index to each key so that its associated value can be stored in an array. The index can be obtained through the use of a hash function; Python implements hash functions for several data types. As a demonstration, the generic function to obtain hash codes is hash.

In [5]:
hash("hello")
# Result: -1182655621190490452
# To restrict the number to be a certain range you can use the modulo (%) operator
hash("hello") % 10
# Result: 8

8

Hash maps can be tricky to implement because they need to handle collisions that happen when two different objects have the same hash code. However, all the complexity is elegantly hidden behind the implementation and the default collision resolution works well in most real-world scenarios

Access, insertion, and removal of an item in a dictionary scales as O(1) with the size of the dictionary

A dictionary can be used to efficiently count unique elements in a list. In this example, we define the counter_dict function that takes a list and returns a dictionary containing the number of occurrences of each value in the list

*counter_dict(items), O(N)*

In [ ]:
def counter_dict(items):
    counter = {}
    for item in items:
        if item not in counter:
            counter[item] = 0
        else:
            counter[item] += 1
    return counter

In the following code, the defaultdict(int) call produces a dictionary where every new element is automatically assigned a zero value, and can be used to streamline the counting:

*counter_defaultdict(items), O(N)*

In [ ]:
from collections import defaultdict
def counter_defaultdict(items):
    counter = defaultdict(int)
    for item in items:
        counter[item] += 1
    return counter

*Counter(items), O(N)*

In [ ]:
from collections import Counter
counter = Counter(items)

#### Building an in-memory search index using a hash map

Dictionaries can be used to quickly search for a word in a list of documents, similar to a search engine

A simple way to retrieve all the documents that match a query is to scan each document and test for the presence of a word

per-query cost of the linear scan is O(N)

In [10]:
docs = ["the cat is under the table",
        "the dog is under the table",
        "cats and dogs smell roses",
        "Carla eats an apple"]

matches = [doc for doc in docs if "table" in doc]
matches

['the cat is under the table', 'the dog is under the table']

Build a structure, called the inverted index, that associates each word in our collection with the list of documents where that word is present. In our earlier example, the word "table" will be associated to the "the cat is under the table" and "the dog is under the table" documents; they correspond to indices 0 and 1.

In [13]:
# Building an index
index = {}
for i, doc in enumerate(docs):
    # We iterate over each term in the document
    for word in doc.split():
        # We build a list containing the indices where the term appears
        if word not in index:
            index[word] = [i]
        else:
            index[word].append(i)
            
results = index['table']
result_documents = [docs[i] for i in results]
result_documents

['the cat is under the table', 'the dog is under the table']

Since all it takes to query our collection is a dictionary access, the index can handle queries with time complexity O(1)

### Sets

Sets are unordered collections of elements, with the additional restriction that the elements must be unique

Sets are implemented using a hash-based algorithm just like dictionaries, the time complexities for addition, deletion, and test for membership scale as O(1) with the size of the collection

Sets used for duplicates removal

In [ ]:
# create a list that contains duplicates
x = list(range(1000)) + list(range(500))
# the set *x_unique* will contain only the unique elements in x
x_unique = set(x)

The time complexity for removing duplicates is O(N), as it requires to read the input and put each element in the set.

*s.union(t), O(S+T)*

*s.intersection(t), O(min(S,T))*

*s.difference(t), O(S)*

In [14]:
# Building an index using sets
index = {}
for i, doc in enumerate(docs):
    # We iterate over each term in the document
    for word in doc.split():
        # We build a set containing the indices where the term appears
        if word not in index:
            index[word] = {i}
        else:
            index[word].add(i)
            
# Querying the documents containing both "cat" and "table"
index['cat'].intersection(index['table'])

{0}

### Heaps

Heaps are data structures designed to quickly find and extract the maximum (or minimum) value in a collection. A typical use-case for heaps is to process a series of incoming tasks in order of maximum priority

A heap is a more efficient data structure that allows for insertion and extraction of maximum values with O(log(N)) time complexity.

In [15]:
import heapq

collection = [10, 3, 3, 4, 5, 6]
heapq.heapify(collection)

In [16]:
heapq.heappop(collection)
# Returns: 3

3

In [17]:
heapq.heappush(collection, 1)

In [18]:
collection

[1, 4, 3, 10, 5, 6]

Extract the minimum value in the collection

In [19]:
from queue import PriorityQueue

queue = PriorityQueue()
for element in collection:
    queue.put(element)
    
queue.get()
# Returns: 3

1

In [20]:
queue = PriorityQueue()
queue.put((3, "priority 3"))
queue.put((2, "priority 2"))
queue.put((1, "priority 1"))
queue.get()
# Returns: (1, "priority 1")

(1, 'priority 1')

### Tries

Tries are extremely fast at matching a list of strings against a prefix. This is especially useful when implementing features such as search-as-you type and autocompletion, where the list of available completions is very large and short response times are required

Perform the task of finding the longest prefix in a set of strings (just like autocompletion).

In [ ]:
from random import choice
from string import ascii_uppercase

def random_string(length):
"""Produce a random string made of *length* uppercase ascii
characters"""
return ''.join(choice(ascii_uppercase) for i in range(length))

In [ ]:
strings = [random_string(32) for i in range(10000)]
matches = [s for s in strings if s.startswith('AA')]

In [ ]:
%timeit [s for s in strings if s.startswith('AA')]

In [ ]:
pip install patricia-trie

In [ ]:
from patricia import trie
strings_dict = {s:0 for s in strings}
# A dictionary where all values are 0
strings_trie = trie(**strings_dict)

In [ ]:
matches = list(strings_trie.iter('AA'))

In [ ]:
%timeit list(strings_trie.iter('AA'))

Querying a trie has a time complexity O(S), where S is the length of the longest string in the collection, while the time complexity of a simple linear scan is O(N), where N is the size of the collection

## 2.2 Caching and memoization

Caching is a great technique used to improve the performance of a wide range of applications. The idea behind caching is to store expensive results in a temporary location, called cache, that can be located in memory, on-disk, or in a remote location.

Web applications make extensive use of caching. In a web application, it often happens that users request a certain page at the same time. In this case, instead of recomputing the page for each user, the web application can compute it once and serve the user the already rendered page. Ideally, caching also needs a mechanism for invalidation so that if the page needs to be updated, we can recompute it before serving it again. Intelligent caching allows web applications to handle increasing number of users with less resources. Caching can also be done preemptively, such as the later sections of the video get buffered when watching a video online.

Storing and reusing the results of the previous function calls in an application is usually termed as **memoization**, and is one of the forms of caching. Several other algorithms can take advantage of memoization to gain impressive performance improvements, and this programming technique is commonly referred to as **dynamic programming**.

The benefits of caching, however, do not come for free. What we are actually doing is sacrificing some space to improve the speed of the application. Additionally, if the cache is stored in a location on the network, we may incur transfer costs and general time needed for communication. One should evaluate when it is convenient to use a cache and how much space we are willing to trade for an increase in speed

Python standard library includes a simple inmemory cache out of the box in the *functools* module

In [27]:
from functools import lru_cache

@lru_cache()
def sum2(a, b):
    print("Calculating {} + {}".format(a, b))
    return a + b

In [28]:
print(sum2(1, 2))
# Output:
# Calculating 1 + 2
# 3

Calculating 1 + 2
3


In [29]:
print(sum2(1, 2))
# Output:
# 3

3


In [ ]:
@lru_cache(max_size=16)
def sum2(a, b):

In [ ]:
sum2.cache_info()
# Output: CacheInfo(hits=0, misses=1, maxsize=128, currsize=1)
sum2.cache_clear()

In [31]:
def fibonacci(n):
    if n < 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)
    
# Non-memoized version
%timeit fibonacci(20)
# 100 loops, best of 3: 5.57 ms per loop

2.88 ms ± 27.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


the previously computed fibonacci sequences are not reused, causing this algorithm to have an exponential scaling of roughly O(2N).

Caching can improve this algorithm by storing and reusing the already-computed fibonacci numbers.

In [32]:
import timeit
setup_code = '''
from functools import lru_cache
from __main__ import fibonacci
fibonacci_memoized = lru_cache(maxsize=None)(fibonacci)
'''

results = timeit.repeat('fibonacci_memoized(20)',
                        setup=setup_code,
                        repeat=1000,
                        number=1)

print("Fibonacci took {:.2f} us".format(min(results)))
# Output: Fibonacci took 0.01 us

Fibonacci took 0.00 us


### Joblib

joblib is on-disk cache. The package can be used in a similar way as lru_cache, except that the results will be stored on disk and will persist between runs

In [ ]:
conda update joblib

In [ ]:
from joblib import Memory
memory = Memory(cachedir='/path/to/cachedir')

@memory.cache
def sum2(a, b):
    return a + b

The function will behave similar to lru_cache, with the exception that the results will be stored on-disk in the directory specified by the cachedir argument during Memory initialization. Additionally, the cached results will persist over subsequent runs.

Perhaps the best joblib feature is that, thanks to intelligent hashing algorithms, it provides efficient memoization of functions that operate on numpy arrays, and is particularly useful in scientific and engineering applications

## 2.3 Comprehensions and generators 

In [2]:
def loop():
    res = []
    for i in range(100000):
        res.append(i * i)
    return sum(res)

def comprehension():
    return sum([i * i for i in range(100000)])

def generator():
    return sum(i * i for i in range(100000))

%timeit loop()
# 100 loops, best of 3: 16.1 ms per loop
%timeit comprehension()
# 100 loops, best of 3: 10.1 ms per loop
%timeit generator()
# 100 loops, best of 3: 12.4 ms per loop

8.57 ms ± 88.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.64 ms ± 49.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.41 ms ± 94.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
def loop():
    res = {}
    for i in range(100000):
        res[i] = i
    return res

def comprehension():
    return {i: i for i in range(100000)}

%timeit loop()
# 100 loops, best of 3: 13.2 ms per loop
%timeit comprehension()
# 100 loops, best of 3: 12.8 ms per loop

5.79 ms ± 169 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.03 ms ± 26 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Efficient looping (especially in terms of memory) can be implemented using iterators and functions such as filter and map

In [4]:
def map_comprehension(numbers):
    a = [n * 2 for n in numbers]
    b = [n ** 2 for n in a]
    c = [n ** 0.33 for n in b]
    return max(c)

The problem with this approach is that for every list comprehension, we are allocating a new list, increasing memory usage. Instead of using list comprehension, we can employ generators. Generators are objects that, when iterated upon, compute a value on the fly and return the result

For example, the map function takes two arguments--a function and an iterator--and returns a generator that applies the function to every element of the collection. *The important point is that the operation happens only while we are iterating, and not when map is invoked!*

We can rewrite the previous function using map and by creating intermediate generators, rather than lists, thus saving memory by computing the values on the fly:

In [ ]:
def map_normal(numbers):
    a = map(lambda n: n * 2, numbers)
    b = map(lambda n: n ** 2, a)
    c = map(lambda n: n ** 0.33, b)
    return max(c)

In [ ]:
%load_ext memory_profiler
numbers = range(1000000)
%memit map_comprehension(numbers)
# peak memory: 166.33 MiB, increment: 102.54 MiB
%memit map_normal(numbers)
# peak memory: 71.04 MiB, increment: 0.00 MiB

## 2.4 Summary